In [1]:
#library(IRdisplay)
library(tidyverse)
library(stringr)

── Attaching core tidyverse packages ───────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
df <- read.csv("../input/laptop_price_explored.csv")

In [3]:
head(df)

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
6,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.1kg,400.00


### Data Preparation

#### Feature: Company

In [4]:
company_counts <- table(df$Company)
company_counts


     Acer     Apple      Asus     Chuwi      Dell   Fujitsu    Google        HP 
      103        21       158         3       297         3         3       274 
   Huawei    Lenovo        LG  Mediacom Microsoft       MSI     Razer   Samsung 
        2       297         3         7         6        54         7         9 
  Toshiba      Vero    Xiaomi 
       48         4         4 

In [5]:
companies_to_replace <- names(company_counts[company_counts < 20])

df$Company[df$Company %in% companies_to_replace] <- "Other"

table(df$Company)


   Acer   Apple    Asus    Dell      HP  Lenovo     MSI   Other Toshiba 
    103      21     158     297     274     297      54      51      48 

#### Feature: Product

In [6]:
df$Model <- sapply(strsplit(as.character(df$Product), " "), "[", 1)


num_unique_models <- length(unique(df$Model))

print(num_unique_models)

[1] 225


In [7]:
head(dplyr::count(df, Model, sort=TRUE), 10)

,Model,n
,<chr>,<int>
1,Inspiron,141
2,IdeaPad,95
3,Aspire,63
4,ThinkPad,62
5,Latitude,52
6,ProBook,50
7,EliteBook,45
8,XPS,39
9,Thinkpad,37


#### Feature: Inches

In [8]:
unique(df$Inches)

[1] 13.3 15.6 15.4 14.0 12.0 11.6 17.3 10.1 13.5 12.5 13.0 18.4 13.9 12.3 17.0
[16] 15.0 14.1 11.3

In [9]:
# Define the bin edges
bin_edges <- c(0, 11.8, 12.6, 13.7, 14.2, 16.8, Inf)

# Create custom labels
custom_labels <- c(0, 1, 2, 3, 4, 5)

# Create a new categorical variable based on binning
df$Inches_Binned <- cut(df$Inches, breaks = bin_edges, labels = custom_labels)

In [10]:
dplyr::count(df, Inches_Binned, sort=TRUE)

Inches_Binned,n
<fct>,<int>
4,673
3,204
2,172
5,166
1,50
0,38


#### Feature: Screen Resolution

In [11]:
# Extract the resolution from the "ScreenResolution" column
df$Resolution <- str_extract(df$ScreenResolution, "\\d+x\\d+")

# Create a new column for the display type
df$DisplayType <- str_trim(str_replace(df$ScreenResolution, "\\d+x\\d+", ""))

# Replace empty strings with NA
df$DisplayType[df$DisplayType == ""] <- NA

# Drop the original "ScreenResolution" column
df <- subset(df, select = -c(ScreenResolution))

In [12]:
# Create touchscreen variable
df$Touchscreen <- ifelse(grepl("Touchscreen", df$DisplayType, ignore.case = TRUE), "1", "0")

# Create IPS variable
df$IPS_Panel <- ifelse(grepl("IPS Panel", df$DisplayType, ignore.case = TRUE), "1", "0")

# Create Retina Display variable
df$RetinaDisplay <- ifelse(grepl("Retina Display", df$DisplayType, ignore.case = TRUE), "1", "0")

# Remove "Touchscreen" and "/" from "DisplayType"
df$DisplayType <- gsub("Touchscreen", "", df$DisplayType, ignore.case = TRUE)
df$DisplayType <- gsub("/ ", "", df$DisplayType)
df$DisplayType <- gsub("IPS Panel", "", df$DisplayType, ignore.case = TRUE)
df$DisplayType <- trimws(df$DisplayType)

# Replace '' with NA for visualization
df$DisplayType[df$DisplayType == ""] <- NA
df$Touchscreen[df$Touchscreen == ""] <- NA
df$IPS_Panel[df$IPS_Panel == ""] <- NA
df$RetinaDisplay[df$RetinaDisplay == ""] <- NA



In [16]:
unique(df$Resolution)

[1] "2560x1600" "1440x900"  "1920x1080" "2880x1800" "1366x768"  "2304x1440"
 [7] "3200x1800" "1920x1200" "2256x1504" "3840x2160" "2160x1440" "2560x1440"
[13] "1600x900"  "2736x1824" "2400x1600"

In [18]:
# Split the "Resolution" column into "ResolutionWidth" and "ResolutionHeight" columns
df <- df %>%
  separate(Resolution, into = c("ResolutionWidth", "ResolutionHeight"), sep = "x", convert = TRUE)
head(df)

,Company,Product,TypeName,Inches,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Model,Inches_Binned,ResolutionWidth,ResolutionHeight,DisplayType,Touchscreen,IPS_Panel,RetinaDisplay
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,Apple,MacBook Pro,Ultrabook,13.3,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,MacBook,2,2560,1600,Retina Display,0,1,1
2,Apple,Macbook Air,Ultrabook,13.3,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94,Macbook,2,1440,900,NA,0,0,0
3,HP,250 G6,Notebook,15.6,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,250,4,1920,1080,Full HD,0,0,0
4,Apple,MacBook Pro,Ultrabook,15.4,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45,MacBook,4,2880,1800,Retina Display,0,1,1
5,Apple,MacBook Pro,Ultrabook,13.3,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,MacBook,2,2560,1600,Retina Display,0,1,1
6,Acer,Aspire 3,Notebook,15.6,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.1kg,400.00,Aspire,4,1366,768,NA,0,0,0


#### Feature: Cpu

In [20]:
head(dplyr::count(df, Cpu, sort=TRUE), 10)

,Cpu,n
,<chr>,<int>
1,Intel Core i5 7200U 2.5GHz,190
2,Intel Core i7 7700HQ 2.8GHz,146
3,Intel Core i7 7500U 2.7GHz,134
4,Intel Core i7 8550U 1.8GHz,73
5,Intel Core i5 8250U 1.6GHz,72
6,Intel Core i5 6200U 2.3GHz,68
7,Intel Core i3 6006U 2GHz,64
8,Intel Core i7 6500U 2.5GHz,49
9,Intel Core i7 6700HQ 2.6GHz,43
